In [6]:
import numpy as np
from qiskit import QuantumCircuit

# 1. A quantum circuit for preparing the quantum state |000> + i |111> / √2
qc = QuantumCircuit(3)
qc.h(2)             # generate superposition
qc.p(np.pi / 2, 0)  # add quantum phase
qc.cx(0, 1)         # 0th-qubit-Controlled-NOT gate on 1st qubit
qc.cx(0, 2)         # 0th-qubit-Controlled-NOT gate on 2nd qubit
qc.draw()


┌────────┐          
q_0: ┤ P(π/2) ├──■────■──
     └────────┘┌─┴─┐  │  
q_1: ──────────┤ X ├──┼──
       ┌───┐   └───┘┌─┴─┐
q_2: ──┤ H ├────────┤ X ├
       └───┘        └───┘

In [7]:
# 2. Add the classical output in the form of measurement of all qubits
qc_measured = qc.measure_all(inplace=False)

# 3. Execute using the Sampler primitive
from qiskit.primitives import StatevectorSampler
sampler = StatevectorSampler()
job = sampler.run([qc_measured], shots=1000)
result = job.result()
print(f" > Counts: {result[0].data['meas'].get_counts()}")

 > Counts: {'100': 500, '000': 500}


In [12]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.transpiler import CouplingMap

# ----------------------------
# 1) Build your circuit
# ----------------------------
qc = QuantumCircuit(3)
qc.h(2)
qc.p(np.pi / 2, 0)
qc.cx(0, 1)
qc.cx(0, 2)
qc.measure_all()

print("\n=== Original circuit ===")
print(qc)

# ----------------------------
# 2) Backend: local simulator
# ----------------------------
backend = Aer.get_backend("aer_simulator")


# ----------------------------
# Helper: robust mapping printer
# ----------------------------
def print_mapping(original_circuit, transpiled_circuit, title="Mapping"):
    print(f"\n=== {title} ===")

    layout = getattr(transpiled_circuit, "layout", None) or getattr(transpiled_circuit, "_layout", None)
    if layout is None:
        print("No layout object exists (backend didn't need mapping).")
        return

    final_layout = getattr(layout, "final_layout", None)
    initial_layout = getattr(layout, "initial_layout", None)
    chosen_layout = final_layout if final_layout is not None else initial_layout

    if chosen_layout is None:
        print("Layout exists, but both final_layout and initial_layout are None.")
        return

    try:
        vbits = chosen_layout.get_virtual_bits()  # dict: virtual_qubit -> physical_index
    except Exception as e:
        print("Could not extract virtual bits mapping:", e)
        return

    # Only original qubits
    orig_qubits = set(original_circuit.qubits)

    print("Logical -> Physical mapping (original qubits only):")
    printed_any = False
    for vq, pq in vbits.items():
        if vq not in orig_qubits:
            continue  # skip ancillas or extra qubits not in the original circuit
        logical_index = original_circuit.qubits.index(vq)
        print(f"  logical q{logical_index} -> physical q{pq}")
        printed_any = True

    if not printed_any:
        print("  (No original qubits found in layout mapping; mapping may be trivial or stored differently.)")

    # Optional: show ancillas too
    ancillas = [(vq, pq) for vq, pq in vbits.items() if vq not in orig_qubits]
    if ancillas:
        print("\nAncilla / extra qubits introduced by transpiler:")
        for vq, pq in ancillas:
            print(f"  {vq} -> physical q{pq}")


# ----------------------------
# 3) Normal transpile (Aer usually doesn't need layout)
# ----------------------------
qc_hw = transpile(qc, backend=backend, optimization_level=3)

print("\n=== Transpiled circuit (normal) ===")
print(qc_hw)
print_mapping(qc, qc_hw, "Normal transpile mapping")


# ----------------------------
# 4) Forced layout transpile
# ----------------------------
# Map logical q0,q1,q2 -> "physical" indices 5,2,7
forced_layout = [5, 2, 7]

qc_forced = transpile(
    qc,
    backend=backend,
    optimization_level=3,
    initial_layout=forced_layout
)

print("\n=== Transpiled circuit (forced layout) ===")
print(qc_forced)
print_mapping(qc, qc_forced, "Forced layout mapping")


# ----------------------------
# 5) BONUS: transpile with coupling map constraints (hardware-like)
# ----------------------------
# line topology: 0-1-2-3-4
coupling = CouplingMap.from_line(5)

qc_constrained = transpile(
    qc,
    basis_gates=["rz", "sx", "x", "cx"],  # IBM-like basis
    coupling_map=coupling,
    optimization_level=3,
    initial_layout=[0, 1, 2]
)

print("\n=== Transpiled circuit (with coupling map constraints) ===")
print(qc_constrained)
print_mapping(qc, qc_constrained, "Constrained (coupling-map) mapping")


# ----------------------------
# 6) Run simulation counts
# ----------------------------
def run_and_print_counts(circuit, name, shots=1000):
    job = backend.run(circuit, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print(f"\n=== Simulation counts: {name} ===")
    print(counts)

run_and_print_counts(qc_hw, "Normal transpile", shots=1000)
run_and_print_counts(qc_forced, "Forced layout transpile", shots=1000)
run_and_print_counts(qc_constrained, "Coupling-map constrained transpile", shots=1000)




=== Original circuit ===
        ┌────────┐           ░ ┌─┐      
   q_0: ┤ P(π/2) ├──■────■───░─┤M├──────
        └────────┘┌─┴─┐  │   ░ └╥┘┌─┐   
   q_1: ──────────┤ X ├──┼───░──╫─┤M├───
          ┌───┐   └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ──┤ H ├────────┤ X ├─░──╫──╫─┤M├
          └───┘        └───┘ ░  ║  ║ └╥┘
meas: 3/════════════════════════╩══╩══╩═
                                0  1  2 

=== Transpiled circuit (normal) ===
        ┌────────┐           ░ ┌─┐      
   q_0: ┤ P(π/2) ├──■────■───░─┤M├──────
        └────────┘┌─┴─┐  │   ░ └╥┘┌─┐   
   q_1: ──────────┤ X ├──┼───░──╫─┤M├───
          ┌───┐   └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ──┤ H ├────────┤ X ├─░──╫──╫─┤M├
          └───┘        └───┘ ░  ║  ║ └╥┘
meas: 3/════════════════════════╩══╩══╩═
                                0  1  2 

=== Normal transpile mapping ===
No layout object exists (backend didn't need mapping).

=== Transpiled circuit (forced layout) ===
                   ┌───┐      ░    ┌─┐   
q_1 -> 2 ──────────┤ X ├

In [16]:
from qiskit import transpile

qc_u = transpile(qc, basis_gates=["h", "t", "cx"], optimization_level=3)
print(qc_u)
qc_ibm = transpile(qc, basis_gates=["rz", "sx", "x", "cx"], optimization_level=3)
print(qc_ibm)



        ┌───┐┌───┐           ░ ┌─┐      
   q_0: ┤ T ├┤ T ├──■────■───░─┤M├──────
        └───┘└───┘┌─┴─┐  │   ░ └╥┘┌─┐   
   q_1: ──────────┤ X ├──┼───░──╫─┤M├───
        ┌───┐     └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ┤ H ├──────────┤ X ├─░──╫──╫─┤M├
        └───┘          └───┘ ░  ║  ║ └╥┘
meas: 3/════════════════════════╩══╩══╩═
                                0  1  2 
global phase: π/2
        ┌─────────┐                       ░ ┌─┐      
   q_0: ┤ Rz(π/2) ├──■────────────────■───░─┤M├──────
        └─────────┘┌─┴─┐              │   ░ └╥┘┌─┐   
   q_1: ───────────┤ X ├──────────────┼───░──╫─┤M├───
        ┌─────────┐├───┴┐┌─────────┐┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ X ├─░──╫──╫─┤M├
        └─────────┘└────┘└─────────┘└───┘ ░  ║  ║ └╥┘
meas: 3/═════════════════════════════════════╩══╩══╩═
                                             0  1  2 
